In [1]:
from sklearn.model_selection import KFold
import pickle
import pandas as pd
import os.path
file_name = 'resampled_2x5_cross_validation'
if os.path.isfile(file_name+'.pickle'):
    with open(file_name+'.pickle', 'rb') as handle:
        train_folds,test_folds = pickle.load(handle)
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from attentions import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Block(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=None, stride=None):
        super().__init__()
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size)

    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))


class Encoder(nn.Module):
    def __init__(self, chs=(1024, 512, 256, 128, 64), kernel_size=3, stride=None):
        super().__init__()
        self.enc_blocks = nn.ModuleList(
            [Block(chs[i], chs[i + 1], kernel_size=kernel_size, stride=stride) for i in range(len(chs) - 1)])
        self.pool = nn.MaxPool1d(2)

    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x.flatten(1, -1))
            x = self.pool(x)
        return ftrs


class RnnType:
    GRU = 1
    LSTM = 2


class AttentionModel:
    NONE = 0
    DOT = 1
    GENERAL = 2
    CONCAT = 3


class Parameters:
    def __init__(self, data_dict):
        for k, v in data_dict.items():
            exec("self.%s=%s" % (k, v))




    def forward(self, input):
        hidden = self.init_hidden(input.shape[0])
        output, hidden = self.net(input,
                                  hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        if self.method == 'GRU':
            return torch.zeros(self.n_layers * self.n_directions,
                               batch_size,
                               self.hidden_size).to(self.device)
        elif self.method == 'LSTM':
            return (torch.zeros(self.n_layers * self.n_directions,
                                batch_size,
                                self.hidden_size).to(self.device),
                    torch.zeros(self.n_layers * self.n_directions,
                                batch_size,
                                self.hidden_size).to(self.device))
        else:
            raise Exception('Unknown rnn_type. Valid options: "gru", "lstm"')


class Decoder(nn.Module):
    def __init__(self, device, params):
        super(Decoder, self).__init__()
        self.params = params
        self.device = device

        # Calculate number of directions
        self.num_directions = 2 if self.params.bidirectional == True else 1
        self.encoder = Encoder(chs=self.params.chs_encoder,
                               kernel_size=self.params.kernel_size,
                               stride=self.params.stride)
        # Attention layer
        self.attention = MultiHeadAttention(d_model=self.params.rnn_hidden_dim * self.num_directions, num_heads=4)

        # RNN layer
        rnn = None
        if self.params.rnn_type == RnnType.GRU:
            rnn = nn.GRU
            self.decoder_hidden = nn.Sequential(
                torch.nn.LazyLinear(self.params.layer_width),
                torch.nn.ReLU(),
                torch.nn.Linear(self.params.layer_width,
                                self.num_directions * self.params.num_layers * self.params.rnn_hidden_dim),
                torch.nn.Sigmoid()
            )
        elif self.params.rnn_type == RnnType.LSTM:
            rnn = nn.LSTM
            self.decoder_hidden1 = nn.Sequential(
                torch.nn.LazyLinear(self.params.layer_width),
                torch.nn.ReLU(),
                torch.nn.Linear(self.params.layer_width,
                                self.num_directions * self.params.num_layers * self.params.rnn_hidden_dim),
                torch.nn.Sigmoid()
            )
            self.decoder_hidden2 = nn.Sequential(
                torch.nn.LazyLinear(self.params.layer_width),
                torch.nn.ReLU(),
                torch.nn.Linear(self.params.layer_width,
                                self.num_directions * self.params.num_layers * self.params.rnn_hidden_dim),
                torch.nn.Sigmoid()
            )
        else:
            raise Exception("[Error] Unknown RnnType. Currently supported: RnnType.GRU=1, RnnType.LSTM=2")
        self.rnn = rnn(input_size=self.params.rnn_hidden_dim,
                       hidden_size=self.params.rnn_hidden_dim,
                       num_layers=self.params.num_layers,
                       bidirectional=self.params.bidirectional,
                       dropout=self.params.dropout,
                       batch_first=True)
        # self.rnn2 = rnn(input_size=self.params.rnn_hidden_dim*self.num_directions,
        #                hidden_size=self.params.rnn_hidden_dim,
        #                num_layers=self.params.num_layers,
        #                bidirectional=self.params.bidirectional,
        #                dropout=self.params.dropout,
        #                batch_first=True)

        self.decoder_input = nn.Sequential(
            torch.nn.LazyLinear(self.params.layer_width),
            torch.nn.ReLU(),
            torch.nn.Linear(self.params.layer_width, self.params.output_sequence_length * self.params.rnn_hidden_dim),
            torch.nn.Sigmoid()
        )
        self.linear = nn.Sequential(
            torch.nn.Linear(self.params.rnn_hidden_dim * self.num_directions, self.params.layer_width),
            torch.nn.ReLU(),
            torch.nn.Linear(self.params.layer_width, 1),
            torch.nn.Sigmoid()
        )
        self.encoder_output_linear = nn.Sequential(
            nn.LazyLinear(self.params.output_sequence_length*self.params.rnn_hidden_dim*2)
        )
        self.to(device)

    def init_hidden(self, encoder_outputs):
        if self.params.rnn_type == RnnType.GRU:
            hidden = self.decoder_hidden(encoder_outputs).view(self.batch_size,
                                                               self.params.num_layers * self.num_directions,
                                                               self.params.rnn_hidden_dim).transpose(0, 1)
            return hidden.contiguous().to(self.device)
        elif self.params.rnn_type == RnnType.LSTM:
            hidden1 = self.decoder_hidden1(encoder_outputs).view(self.batch_size,
                                                                 self.params.num_layers * self.num_directions,
                                                                 self.params.rnn_hidden_dim).transpose(0, 1)
            hidden2 = self.decoder_hidden2(encoder_outputs).view(self.batch_size,
                                                                 self.params.num_layers * self.num_directions,
                                                                 self.params.rnn_hidden_dim).transpose(0, 1)
            return (hidden1.contiguous().to(self.device),
                    hidden2.contiguous().to(self.device)
                    )
        else:
            raise Exception('Unknown rnn_type. Valid options: "gru", "lstm"')

    def forward(self, inputs):
        self.batch_size, seq_len = inputs.shape  # to encoder
        encoder_outputs = self.encoder(inputs.unsqueeze(1))
        encoder_outputs = torch.cat(encoder_outputs, 1)
        # encoder_outputs (batch_size, N)
        # decoder_inputs: (batch_size, sequence length, hidden)
        decoder_inputs = self.decoder_input(encoder_outputs).view(self.batch_size,
                                                                  self.params.output_sequence_length,
                                                                  self.params.rnn_hidden_dim)
        
        self.hidden = self.init_hidden(encoder_outputs)
        
        # Push through RNN layer
        rnn_output, _ = self.rnn(decoder_inputs, self.hidden)

        # (batch_size, output_seq_len, num_directions*hidden)
        decoder_inputs_ = torch.cat([decoder_inputs, decoder_inputs], 2)
        
        encoder_output_ = self.encoder_output_linear(encoder_outputs).view(self.batch_size,
                                                                  self.params.output_sequence_length,
                                                                  self.params.rnn_hidden_dim*2)
        

        context, att = self.attention(rnn_output, encoder_output_, encoder_output_)
        #context, att = self.attention(rnn_output, decoder_inputs_, decoder_inputs_)

        #context = self.rnn2(context, self.hidden)[0]
        #residual connection
        #context = context + rnn_output
        # print(rnn_output.shape)
        spectrum_out = self.linear(context).squeeze(2) + rnn_output.mean(dim=2)
        return spectrum_out


class XSigmoidLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        # return torch.mean(2 * ey_t / (1 + torch.exp(-ey_t)) - ey_t)
        return torch.mean(2 * ey_t * torch.sigmoid(ey_t) - ey_t)

from torch.utils.data import DataLoader
from torch import optim
import numpy as np
from tqdm import tqdm
class CalculateMSE():
    def __init__(self, net,n_epochs,batch_size ):
        super().__init__()
        self.net = net
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = torch.utils.data.TensorDataset(
            torch.Tensor(train_data),
            torch.Tensor(train_label))
        val_set = torch.utils.data.TensorDataset(
            torch.Tensor(test_data),
            torch.Tensor(test_label))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        tloss = []
        vloss = []
        criterion = XSigmoidLoss()
        default_criterion = nn.MSELoss()
        #add weight decay
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate, weight_decay=0) # weight_decay=5e-4
        # optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        with tqdm(range(0, self.n_epochs)) as pbar:
            for epoch in pbar:
                epoch_train_loss=[]
                for i, data in enumerate(train_loader, 0):
                    inputs, label = data
                    y_pred = self.net(inputs.to(self.net.device))
                    loss = criterion(y_pred, label.to(self.net.device))
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    epoch_train_loss.append(default_criterion(y_pred, label.to(self.net.device)).item())
                tloss.append(np.mean(epoch_train_loss))
                epoch_loss=[]
                for i, data in enumerate(val_loader, 0):
                    with torch.no_grad():
                        inputs1, label1 = data
                        y_pred1 = self.net(inputs1.to(self.net.device))
                        loss1 = default_criterion(y_pred1, label1.to(self.net.device))
                        epoch_loss.append(loss1.item())
                vloss.append(np.mean(epoch_loss))
                pbar.set_postfix({'EPOCH':epoch,
                      'tr_loss':tloss[-1],
                      'val_loss':vloss[-1]})
        return np.min(vloss), self.net



In [2]:
from pathlib import Path
n_epochs=3000
batch_size=32
PATH = 'saved_model/conv_seq2seq_new10_combined_poisson_20percent/'
Path(PATH).mkdir(parents=True, exist_ok=True)
params = {
    'chs_encoder':(1,32,64),  #tuple of channels/feature dimensions
    'kernel_size':4,
    'stride':1,
    'rnn_type':RnnType.GRU,
    'rnn_hidden_dim': 32,
    'layer_width':2000,
    'num_layers':1,
    'dropout':0,
    'attention_model':AttentionModel.GENERAL,
    'sequence_length':64,
    'output_sequence_length':250,
    'batch_size':32,
    'bidirectional':True}
params  =Parameters(params)
losses = []


In [3]:
def add_noise(inputs,inputs2, a = 0.02, std = 0.02, sequence_length=64, noise_seed=None):
    if noise_seed is not None:
        np.random.seed(noise_seed)
    noise = np.random.normal(0,std, size = (inputs.shape[0], sequence_length)).astype(np.float32)
    noise2 = np.random.normal(0,std, size = (inputs2.shape[0], sequence_length)).astype(np.float32)
    #noise = poisson.rvs(mu, size=(inputs.shape[0], sequence_length)).astype(np.float32)
    
    nsd = a*np.random.poisson(inputs/a).astype(np.float32)
    nsd2 = a*np.random.poisson(inputs2/a).astype(np.float32)
    
    # Calculate the absolute error between nsd and original inputs
    absolute_error = np.abs(nsd - inputs)
    absolute_error2 = np.abs(nsd2 - inputs2)
    # Sum up the absolute errors
    total_error = np.sum(absolute_error)
    total_error2 = np.sum(absolute_error2)
    # Sum of the original data
    total_original = np.sum(inputs)
    total_original2 = np.sum(inputs2)

    # Calculate the noise ratio
    noise_ratio = (total_error+total_error2) / (total_original+total_original2)
    
    return nsd, nsd2, noise_ratio
noise_ratios = []
for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    # Adding noise to the train and test data
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.026,std = 0.05, sequence_length=64, noise_seed=i)
    print(noise_ratio)
    noise_ratios.append(noise_ratio)
    mdl =  Decoder(device,params)
    mse_calculator = CalculateMSE(mdl,n_epochs,batch_size)
    loss,model = mse_calculator.get_mse(train_data, train_label, test_data, test_label)
    print(i,loss)
    losses.append(loss)
    torch.save(model.state_dict(), PATH+'model'+str(i))
print(np.mean(losses),np.std(losses))
print(np.mean(noise_ratios))

0.2013326918368819


/home/research/junbo.s/.conda/envs/new_env/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 3000/3000 [13:33<00:00,  3.69it/s, EPOCH=2999, tr_loss=5.2e-6, val_loss=0.0184]  


0 0.012557580322027206


/home/research/junbo.s/.conda/envs/new_env/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


0.19977448149890295


...

7 0.011764382570981979
0.20072605348301625


100%|██████████| 3000/3000 [13:43<00:00,  3.64it/s, EPOCH=2999, tr_loss=1.4e-5, val_loss=0.0171]  


8 0.013531330227851867
0.1999563600361216


100%|██████████| 3000/3000 [13:25<00:00,  3.73it/s, EPOCH=2999, tr_loss=8.1e-6, val_loss=0.0289]  


9 0.01960822716355324
0.015894292714074252 0.0027104879603865287
0.20056441992655402


In [5]:
from scipy import stats
import dcor
#pip install dtw-python
from dtw import *
import torch
import numpy as np

from pathlib import Path
n_epochs=3000
batch_size=32
mdl =  Decoder(device,params)

correlation_losses = []
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)
    construction = model(test_data_tensor).detach().cpu().numpy()
   
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    
    # Distance Correlation
    distance_corr = []
    
    #DTW distance
    alignment = []
    
    #absolute_error
    abs_err = []
    
    for i in range(test_label.shape[0]):
        x1 = construction[i,:]
        x2 = test_label[i,:]
        
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(dcor.distance_correlation(x1,x2))
        
        alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results

for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    print(i)
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.026,std = 0.05, sequence_length=64, noise_seed=i)
    mdl_name = PATH + 'model' + str(i)
    mdl.load_state_dict(torch.load(mdl_name))
    mdl.eval()
    
    correlation_loss = calculate_correlation(mdl, test_data, test_label)
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

0
1
2
3
4
5
6
7
8
9
pearson
0.8132896535353398 0.36865967082821144
0.009121825612633208 0.07201307640743256
kendall
0.6485829108848404 0.27968066548882037
0.012405151266421804 0.08818218116669196
spearman
0.7692935051332188 0.3364100037890996
0.012666618997786674 0.08767564652770236
DTW
13.054261039922274 19.37646333707158
Absolute Error
5
0.000869096274737588
50
0.025436672446575243
90
0.21368723113695756
95
0.3651936817657969
99
0.6285821654908152
Distance Correlation
0.8850314657236138 0.18191248375985603


In [6]:
print(np.mean(losses),np.std(losses))

0.015894292714074252 0.0027104879603865287
